<a href="https://colab.research.google.com/github/chunribu/biotable/blob/main/src/pandas_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
__author__ = "@chunribu[GitHub]"

Welcome🖐!

Let's focus on [pandas](https://pandas.pydata.org/) in this section, a powerful tool for **data analysis and manipulation**. 

pandas is built on top of the [Python](https://www.python.org/) programming language and this tutorial assume you have learned fundamentals of Python, including items bellow, which will be used later.
+ [String operations](https://docs.python.org/3.10/library/stdtypes.html#string-methods)
+ [List operations](https://docs.python.org/3/tutorial/datastructures.html#more-on-lists)
+ [Comprehensions](https://www.geeksforgeeks.org/comprehensions-in-python/)
+ [Lambda function](https://www.tutorialsteacher.com/python/python-lambda-function)
+ etc.



The goal of this tutorial is to be comprehensible and practical, especially for **biodata processing** like data from NCBI databases.

First of all, there are several high-level concepts you need to know before diving into details.

+ **Method Chaining**: Method chaining is a programmatic style of invoking multiple method calls sequentially with each call performing an action on the same object and returning it. In pandas, most of operations can be chained for after every operation it return a copy of the source Object. To save memory, deep copy only occurs after a manual trigger.
+ **Series**: One-dimensional ndarray with axis labels (including time series). You can think of it as a vector or a row (or column) of a table.
+ **DataFrame**: Two-dimensional, size-mutable, potentially heterogeneous tabular data. It consists of zero or more Series, consider it a common table.
+ **GroupBy**: GroupBy objects are returned by groupby calls: `pandas.DataFrame.groupby()`, `pandas.Series.groupby()`, etc. A groupby operation is to split the data into groups based on some criteria. Pandas objects can be split on any of their axes. The abstract definition of grouping is to provide a mapping of labels to group names.

# Input/output
A lot of types are supported:
+ Pickling
+ Flat file
+ Clipboard
+ Excel
+ JSON
+ HTML
+ XML
+ Latex
+ HDFStore: PyTables (HDF5)
+ Feather
+ Parquet
+ ORC
+ SAS
+ SPSS
+ SQL
+ Google BigQuery
+ STATA

Let's take one of "Flat file", `csv`(comma-separated values), as example. `refseq-genbank.csv` is from BioProject of NCBI, containing 4 columns: 
`Refseq accn`, `Genbank accn`, `Organism name` and `TaxID`

In [2]:
!wget https://ftp.ncbi.nlm.nih.gov/bioproject/refseq-genbank.csv
!head refseq-genbank.csv

--2021-12-26 11:56:02--  https://ftp.ncbi.nlm.nih.gov/bioproject/refseq-genbank.csv
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.228, 130.14.250.12, 2607:f220:41e:250::13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93506 (91K) [text/csv]
Saving to: ‘refseq-genbank.csv’

refseq-genbank.csv  100%[===================>]  91.31K  --.-KB/s    in 0.1s    

2021-12-26 11:56:02 (724 KB/s) - ‘refseq-genbank.csv’ saved [93506/93506]

Refseq accn,Genbank accn,Organism name,TaxID
PRJNA116,PRJNA10719,Arabidopsis thaliana,3702
PRJNA116,PRJNA11796,Arabidopsis thaliana,3702
PRJNA116,PRJNA13191,Arabidopsis thaliana,3702
PRJNA122,PRJNA12269,Oryza sativa Japonica Group,39947
PRJNA122,PRJDB1747,Oryza sativa Japonica Group,39947
PRJNA127,PRJNA13836,Schizosaccharomyces pombe 972h-,284812
PRJNA127,PRJNA20755,Schizosaccharomyces pombe,4896
PRJNA128,PRJNA43747,Saccharomyces cerevisiae

In [3]:
import pandas as pd
df = pd.read_csv('refseq-genbank.csv')

The method `read_csv` is designed for loading a plain text file with specific separator into a `DataFrame`. Parameter `sep` is set default to `,`(comma), you may need to declare `sep='\t'` to load a tabular plain text file which also know as `tsv`.

Notablly, authors of pandas are so thoughtful that they designed pandas detect compressed type by suffix, for example, a compressed `refseq-genbank.csv.gz` can be loaded directly. The same goes for output when using `to_csv`.

In [4]:
df

,Refseq accn,Genbank accn,Organism name,TaxID
0,PRJNA116,PRJNA10719,Arabidopsis thaliana,3702
1,PRJNA116,PRJNA11796,Arabidopsis thaliana,3702
2,PRJNA116,PRJNA13191,Arabidopsis thaliana,3702
3,PRJNA122,PRJNA12269,Oryza sativa Japonica Group,39947
4,PRJNA122,PRJDB1747,Oryza sativa Japonica Group,39947
...,...,...,...,...
1714,PRJNA756971,PRJNA682572,Prionailurus bengalensis,37029
1715,PRJNA758027,PRJDB3949,Aspergillus udagawae,91492
1716,PRJNA758049,PRJDB7449,Aspergillus pseudoviridinutans,1517512
1717,PRJNA759178,PRJNA597580,Colletes gigas,935657


When exporting data, add `.gz` suffix to save storage. By default, `index` is `True` , turn it `False` if you don't need.

In [5]:
df.to_csv('refseq-genbank.csv.gz', index=False)
!ls -lh refseq*

-rw-r--r-- 1 root root 92K Sep  9 08:30 refseq-genbank.csv
-rw-r--r-- 1 root root 38K Dec 26 11:56 refseq-genbank.csv.gz


# Must-know Usages

## Slice a DataFrame

A pair of methods `loc` and `iloc` are related to slicing, which is one of the frequently used operations. `loc` and `iloc` are basicly the same except that `loc` accepts index/column **name(s)** and `iloc` **number(s)**.

Let's select part of `df` we have built before. 

Because the index numbers are also index names by default, let's first set `Genbank accn` column as index. 

> Tip: Remember to assign result to the original variable, pandas doesn't change originals by default.

In [6]:
df = df.set_index('Genbank accn')
df

,Refseq accn,Organism name,TaxID
Genbank accn,,,
PRJNA10719,PRJNA116,Arabidopsis thaliana,3702
PRJNA11796,PRJNA116,Arabidopsis thaliana,3702
PRJNA13191,PRJNA116,Arabidopsis thaliana,3702
PRJNA12269,PRJNA122,Oryza sativa Japonica Group,39947
PRJDB1747,PRJNA122,Oryza sativa Japonica Group,39947
...,...,...,...
PRJNA682572,PRJNA756971,Prionailurus bengalensis,37029
PRJDB3949,PRJNA758027,Aspergillus udagawae,91492
PRJDB7449,PRJNA758049,Aspergillus pseudoviridinutans,1517512


Now make a slice of the first two rows and the first two columns. 

In [7]:
df.iloc[0:2, 0:2] # which is the same as: 
# df.iloc[[0,1], [0,1]]

,Refseq accn,Organism name
Genbank accn,,
PRJNA10719,PRJNA116,Arabidopsis thaliana
PRJNA11796,PRJNA116,Arabidopsis thaliana


In [8]:
df.loc[['PRJNA10719', 'PRJNA11796'], ['Refseq accn', 'Organism name']]

,Refseq accn,Organism name
Genbank accn,,
PRJNA10719,PRJNA116,Arabidopsis thaliana
PRJNA11796,PRJNA116,Arabidopsis thaliana


Use `:` or nothing if you want all rows/columns.

In [9]:
df.loc[['PRJNA10719','PRJNA11796']]# which is the same as:
# df.loc[['PRJNA10719','PRJNA11796'], :]

,Refseq accn,Organism name,TaxID
Genbank accn,,,
PRJNA10719,PRJNA116,Arabidopsis thaliana,3702
PRJNA11796,PRJNA116,Arabidopsis thaliana,3702


When passing in a list, the output is a sliced DataFrame; when passing in one single index, you will get a Series.

In [10]:
df.loc[:, 'TaxID']

Genbank accn
PRJNA10719        3702
PRJNA11796        3702
PRJNA13191        3702
PRJNA12269       39947
PRJDB1747        39947
                ...   
PRJNA682572      37029
PRJDB3949        91492
PRJDB7449      1517512
PRJNA597580     935657
PRJNA736740      32260
Name: TaxID, Length: 1719, dtype: int64

## Filter by condition(s)

Obtain particular rows with condition(s). 

For example, 9606 is the taxonomy ID of human, now filter `df` by `TaxID`:

In [11]:
df[df.TaxID==9606]# which is the same as:
# df[df['TaxID']==9606]

,Refseq accn,Organism name,TaxID
Genbank accn,,,
PRJNA31257,PRJNA168,Homo sapiens,9606
PRJNA176729,PRJNA178030,Homo sapiens,9606


`df.TaxID==9606` returns a Series of True/False, and the corresponding rows with `True` are selected.

If you want the opposite instead of rows meeting a condition, use `~`:

In [12]:
df[~(df.TaxID!=9606)]

,Refseq accn,Organism name,TaxID
Genbank accn,,,
PRJNA31257,PRJNA168,Homo sapiens,9606
PRJNA176729,PRJNA178030,Homo sapiens,9606


How about more than one conditions?

Just use `&` or `|` to connect multi conditions:

In [13]:
df[(df.TaxID==9606) & (df['Refseq accn']=='PRJNA168')]

,Refseq accn,Organism name,TaxID
Genbank accn,,,
PRJNA31257,PRJNA168,Homo sapiens,9606


Sometimes I just want to know how many there are, then the method `value_counts` can help. It returns a Series of counts of unique values.

In [14]:
(df.TaxID==9606).value_counts()

False    1717
True        2
Name: TaxID, dtype: int64

By the way, `value_counts` can also be used to see how many unique values there are. See the length. And values are sorted by counts.

In [15]:
df['Organism name'].value_counts()

Canis lupus familiaris      4
Gallus gallus               4
Cricetulus griseus          3
Tursiops truncatus          3
Drosophila                  3
                           ..
Streptococcus suis BM407    1
Tectona grandis             1
Diodon liturosus            1
Charadrius vociferus        1
Manis pentadactyla          1
Name: Organism name, Length: 1600, dtype: int64

`pandas.Series.str` can be used to access the values of the series as strings and apply several methods to it. 

> Tip: Series.str is very useful and it's worth your time to dig deeper.


In [16]:
# filter all records related to Escherichia
df[ df['Organism name'].str.contains('Escherichia') ]

,Refseq accn,Organism name,TaxID
Genbank accn,,,
PRJNA225,PRJNA57779,Escherichia coli str. K-12 substr. MG1655,511145
PRJNA226,PRJNA57781,Escherichia coli O157:H7 str. Sakai,386585
PRJNA20733,PRJNA59039,Escherichia coli chi7122,475609
PRJNA33411,PRJNA59381,Escherichia coli IAI39,585057
PRJNA31349,PRJNA62973,Escherichia coli Vir68,563770
PRJNA32773,PRJNA62977,Escherichia coli,562
PRJNA33415,PRJNA62981,Escherichia coli UMN026,585056
PRJNA41221,PRJNA161987,Escherichia coli O83:H1 str. NRG 857C,685038
PRJNA81095,PRJNA176127,Escherichia coli O104:H4 str. 2011C-3493,1133852


## Change value(s)

To change value(s) within a DataFrame, remember to use `loc` and `iloc`.

In [17]:
df_cv = df.reset_index().head().copy()
df_cv

,Genbank accn,Refseq accn,Organism name,TaxID
0,PRJNA10719,PRJNA116,Arabidopsis thaliana,3702
1,PRJNA11796,PRJNA116,Arabidopsis thaliana,3702
2,PRJNA13191,PRJNA116,Arabidopsis thaliana,3702
3,PRJNA12269,PRJNA122,Oryza sativa Japonica Group,39947
4,PRJDB1747,PRJNA122,Oryza sativa Japonica Group,39947


In [18]:
# This is not recommended, and not always work.
df_cv['TaxID'][0] = 123
df_cv

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Genbank accn,Refseq accn,Organism name,TaxID
0,PRJNA10719,PRJNA116,Arabidopsis thaliana,123
1,PRJNA11796,PRJNA116,Arabidopsis thaliana,3702
2,PRJNA13191,PRJNA116,Arabidopsis thaliana,3702
3,PRJNA12269,PRJNA122,Oryza sativa Japonica Group,39947
4,PRJDB1747,PRJNA122,Oryza sativa Japonica Group,39947


In [19]:
# Use loc/iloc. There won't be any warning.
df_cv.loc[0, 'TaxID'] = 456
df_cv

,Genbank accn,Refseq accn,Organism name,TaxID
0,PRJNA10719,PRJNA116,Arabidopsis thaliana,456
1,PRJNA11796,PRJNA116,Arabidopsis thaliana,3702
2,PRJNA13191,PRJNA116,Arabidopsis thaliana,3702
3,PRJNA12269,PRJNA122,Oryza sativa Japonica Group,39947
4,PRJDB1747,PRJNA122,Oryza sativa Japonica Group,39947


How to add a new column?

In [20]:
df_cv['new_col'] = [1,2,3,4,5]
df_cv

,Genbank accn,Refseq accn,Organism name,TaxID,new_col
0,PRJNA10719,PRJNA116,Arabidopsis thaliana,456,1
1,PRJNA11796,PRJNA116,Arabidopsis thaliana,3702,2
2,PRJNA13191,PRJNA116,Arabidopsis thaliana,3702,3
3,PRJNA12269,PRJNA122,Oryza sativa Japonica Group,39947,4
4,PRJDB1747,PRJNA122,Oryza sativa Japonica Group,39947,5


How to add a new row?

In [21]:
df_cv.loc[5] = [5,5,5,5,5]
df_cv

,Genbank accn,Refseq accn,Organism name,TaxID,new_col
0,PRJNA10719,PRJNA116,Arabidopsis thaliana,456,1
1,PRJNA11796,PRJNA116,Arabidopsis thaliana,3702,2
2,PRJNA13191,PRJNA116,Arabidopsis thaliana,3702,3
3,PRJNA12269,PRJNA122,Oryza sativa Japonica Group,39947,4
4,PRJDB1747,PRJNA122,Oryza sativa Japonica Group,39947,5
5,5,5,5,5,5


## Traverse

How to traverse each row of a DataFrame?
Use `iterrows` method.

In [22]:
for index, row in df.iterrows():
    print(f'------------- {index} -------------')
    print(row)
    break

------------- PRJNA10719 -------------
Refseq accn                  PRJNA116
Organism name    Arabidopsis thaliana
TaxID                            3702
Name: PRJNA10719, dtype: object


How to traverse each column of a DataFrame?

In [23]:
for i in df:
    print('>> i:\n', i)
    print('>> df[i]:\n', df[i])
    break

>> i:
 Refseq accn
>> df[i]:
 Genbank accn
PRJNA10719        PRJNA116
PRJNA11796        PRJNA116
PRJNA13191        PRJNA116
PRJNA12269        PRJNA122
PRJDB1747         PRJNA122
                  ...     
PRJNA682572    PRJNA756971
PRJDB3949      PRJNA758027
PRJDB7449      PRJNA758049
PRJNA597580    PRJNA759178
PRJNA736740    PRJNA759270
Name: Refseq accn, Length: 1719, dtype: object


Use `apply` method, not FOR loop. It accepts a function that appled to every column in a DataFrame by default.

In [24]:
df_cv['new_col'].apply(lambda x: x**2)

0     1
1     4
2     9
3    16
4    25
5    25
Name: new_col, dtype: int64

`apply` method is not efficient enough, try pandas's native solution if there is. 
For example, `isin` is used to assert if the element is in a given set or not. That runs faster than `in` operation in Python.

In [25]:
df.TaxID.isin([9606])

Genbank accn
PRJNA10719     False
PRJNA11796     False
PRJNA13191     False
PRJNA12269     False
PRJDB1747      False
               ...  
PRJNA682572    False
PRJDB3949      False
PRJDB7449      False
PRJNA597580    False
PRJNA736740    False
Name: TaxID, Length: 1719, dtype: bool

> Tip: Efficiency is one of the factors that people choose pandas, using `apply` is a good start to thinking about how to make pandas more efficient.

## Combine datasets

Let's download another table named "summary.txt" form NCBI BioProject, which provides more informations other than taxonomy and Genbank accession.

The goal is to merge `df` and `df_sum`, in other words, match rows by common keys which is "Genbank accn" in `df` and "Project Accession" in `df_sum`.

In [26]:
!wget https://ftp.ncbi.nlm.nih.gov/bioproject/summary.txt
!ls -lh

--2021-12-26 11:56:03--  https://ftp.ncbi.nlm.nih.gov/bioproject/summary.txt
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.10, 2607:f220:41e:250::13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52395690 (50M) [text/plain]
Saving to: ‘summary.txt’

summary.txt         100%[===================>]  49.97M  34.1MB/s    in 1.5s    

2021-12-26 11:56:05 (34.1 MB/s) - ‘summary.txt’ saved [52395690/52395690]

total 51M
-rw-r--r-- 1 root root  92K Sep  9 08:30 refseq-genbank.csv
-rw-r--r-- 1 root root  38K Dec 26 11:56 refseq-genbank.csv.gz
drwxr-xr-x 1 root root 4.0K Dec  3 14:33 sample_data
-rw-r--r-- 1 root root  50M Dec 25 16:10 summary.txt


In [27]:
df_sum = pd.read_csv('summary.txt', sep='\t')
df_sum

,Organism Name,TaxID,Project Accession,Project ID,Project Type,Project Data Type,Date
0,Borreliella burgdorferi B31,224326,PRJNA3,3,Primary submission,Genome sequencing,2003/02/23
1,Treponema denticola ATCC 35405,243275,PRJNA4,4,Primary submission,Genome sequencing,2004/02/03
2,Treponema pallidum subsp. pallidum str. Nichols,243276,PRJNA5,5,Primary submission,Genome sequencing,2003/02/25
3,Campylobacter fetus subsp. venerealis str. Azu...,593452,PRJNA7,7,Primary submission,Genome sequencing and assembly,2009/04/22
4,Campylobacter jejuni subsp. jejuni NCTC 11168 ...,192222,PRJNA8,8,Primary submission,Genome sequencing,2003/05/06
...,...,...,...,...,...,...,...
558043,Pisum sativum L.,3888,PRJNA792203,792203,Primary submission,Transcriptome or Gene expression,2021/12/25
558044,Multispecies,0,PRJNA792206,792206,Primary submission,Raw sequence reads,2021/12/25
558045,Multispecies,0,PRJNA792210,792210,Primary submission,Raw sequence reads,2021/12/25
558046,Thermus sp.,275,PRJNA792213,792213,Primary submission,Genome sequencing and assembly,2021/12/25


In [28]:
df.merge(df_sum, left_on='Genbank accn', right_on='Project Accession', suffixes=('_left','_right'))

,Refseq accn,Organism name,TaxID_left,Organism Name,TaxID_right,Project Accession,Project ID,Project Type,Project Data Type,Date
0,PRJNA116,Arabidopsis thaliana,3702,Arabidopsis thaliana,3702,PRJNA10719,10719,Primary submission,Genome sequencing,2011/05/05
1,PRJNA116,Arabidopsis thaliana,3702,Arabidopsis thaliana,3702,PRJNA11796,11796,Primary submission,Genome sequencing,2004/06/23
2,PRJNA116,Arabidopsis thaliana,3702,Arabidopsis thaliana,3702,PRJNA13191,13191,Primary submission,Genome sequencing,2004/11/19
3,PRJNA122,Oryza sativa Japonica Group,39947,Oryza sativa Japonica Group,39947,PRJNA12269,12269,Primary submission,Genome sequencing,2008/02/19
4,PRJNA122,Oryza sativa Japonica Group,39947,Oryza sativa Japonica Group,39947,PRJDB1747,245403,Primary submission,Genome sequencing,2015/10/10
...,...,...,...,...,...,...,...,...,...,...
1714,PRJNA756971,Prionailurus bengalensis,37029,Prionailurus bengalensis,37029,PRJNA682572,682572,Primary submission,Genome sequencing and assembly,2020/12/16
1715,PRJNA758027,Aspergillus udagawae,91492,Aspergillus udagawae,91492,PRJDB3949,288122,Primary submission,Genome sequencing and assembly,2015/06/24
1716,PRJNA758049,Aspergillus pseudoviridinutans,1517512,Aspergillus pseudoviridinutans,1517512,PRJDB7449,714204,Primary submission,Genome sequencing and assembly,2021/03/12
1717,PRJNA759178,Colletes gigas,935657,Colletes gigas,935657,PRJNA597580,597580,Primary submission,Genome sequencing and assembly,2020/05/19


> Tip: Compare 4 different methods in pandas for data merging/joining including `merge`, `join`, `append` and `concat`.

Congratulations! 🎉 

You have learned the essentials to use pandas. Please [continue](https://github.com/chunribu/biotable#applications)!